In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import polars as pl
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import kaggle_evaluation.default_inference_server

In [ ]:
model_rf = None
model_lstm = None
scaler = None
feature_cols = None
history_features = pd.DataFrame()
history_returns = []
window = 15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rf_weight = 0.6
lstm_weight = 0.4

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)

    def forward(self, x):
        _, (h, _) = self.lstm(x)
        return self.sigmoid(self.fc(h[-1]))

In [ ]:
def predict(test: pl.DataFrame) -> float:
    global model_rf, model_lstm, scaler, feature_cols, history_features, history_returns

    if model_rf is None or model_lstm is None:
        train_path = '/kaggle/input/hull-tactical-market-prediction/train.csv'
        if not os.path.exists(train_path):
            raise FileNotFoundError(f"Training file not found at {train_path}")

        train = pd.read_csv(train_path).sort_values('date_id')

        selected_features = ['V13', 'M1', 'S5', 'D1', 'D2', 'M4', 'S2', 'V7', 'M2', 'V10', 'E11', 'E12', 'E7', 'P8', 'M8', 'M12', 'I2', 'P5']
        train['V13_M1_interaction'] = train['V13'] * train['M1']
        train['S5_M4_interaction'] = train['S5'] * train['M4']
        train['V13_S5_interaction'] = train['V13'] * train['S5']
        train['V13_roll_mean'] = train['V13'].rolling(window=5).mean()
        train['S5_roll_mean'] = train['S5'].rolling(window=5).mean()
        train['V13_roll_std'] = train['V13'].rolling(window=5).std()
        train['M4_roll_mean'] = train['M4'].rolling(window=5).mean()
        train['S2_roll_mean'] = train['S2'].rolling(window=5).mean()
        train['I2_roll_mean'] = train['I2'].rolling(window=5).mean()
        train['P8_roll_mean'] = train['P8'].rolling(window=5).mean()
        train['M12_roll_mean'] = train['M12'].rolling(window=5).mean()
        train['P5_roll_mean'] = train['P5'].rolling(window=5).mean()
        for col in ['V13', 'M1', 'S5', 'V7', 'V10', 'M4', 'S2', 'I2', 'M12', 'P8', 'P5', 'forward_returns', 'market_forward_excess_returns']:
            train[f'{col}_lag1'] = train[col].shift(1)
        feature_cols = selected_features + ['V13_M1_interaction', 'S5_M4_interaction', 'V13_S5_interaction', 'V13_roll_mean', 'S5_roll_mean', 'V13_roll_std', 'M4_roll_mean', 'S2_roll_mean', 'I2_roll_mean', 'P8_roll_mean', 'M12_roll_mean', 'P5_roll_mean'] + [f'{col}_lag1' for col in ['V13', 'M1', 'S5', 'V7', 'V10', 'M4', 'S2', 'I2', 'M12', 'P8', 'P5', 'forward_returns', 'market_forward_excess_returns']]

        feature_cols = [col for col in feature_cols if col in train.columns]

        X = train[feature_cols].ffill().bfill().fillna(0)
        y = (train['forward_returns'] > train['risk_free_rate']).astype(int) 

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        model_rf = RandomForestClassifier(n_estimators=15000, max_depth=25, min_samples_leaf=1, random_state=42)
        model_rf.fit(X_scaled, y)

        X_scaled_df = pd.DataFrame(X_scaled, columns=feature_cols)
        sequences = []
        targets = []
        for i in range(window, len(X_scaled_df)):
            sequences.append(X_scaled_df.iloc[i-window:i][feature_cols].values)
            targets.append((train['forward_returns'].iloc[i] > train['risk_free_rate'].iloc[i]).astype(int))
        sequences = np.array(sequences)
        targets = np.array(targets)

        train_size = int(0.8 * len(sequences))
        X_train_seq, X_val_seq = sequences[:train_size], sequences[train_size:]
        y_train_seq, y_val_seq = targets[:train_size], targets[train_size:]

        X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32).unsqueeze(1).to(device)
        X_val_tensor = torch.tensor(X_val_seq, dtype=torch.float32).to(device)
        y_val_tensor = torch.tensor(y_val_seq, dtype=torch.float32).unsqueeze(1).to(device)

        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)

        model_lstm = LSTM(input_size=len(feature_cols), hidden_size=1024, num_layers=2, output_size=1).to(device)
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model_lstm.parameters(), lr=0.002)
        best_val_loss = float('inf')
        patience = 20
        patience_counter = 0

        for epoch in range(300):
            model_lstm.train()
            train_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                output = model_lstm(X_batch)
                output = torch.clamp(output, min=1e-6, max=1-1e-6)
                loss = criterion(output, y_batch)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model_lstm.parameters(), max_norm=1.0)
                optimizer.step()
                train_loss += loss.item()
            
            model_lstm.eval()
            with torch.no_grad():
                val_output = model_lstm(X_val_tensor)
                val_output = torch.clamp(val_output, min=1e-6, max=1-1e-6)
                val_loss = criterion(val_output, y_val_tensor).item()
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    break

        torch.cuda.empty_cache()

    test_pd = test.to_pandas()
    test_features = test_pd[[col for col in feature_cols if col in test_pd.columns]].copy()
    for col in ['V13', 'M1', 'S5', 'V7', 'V10', 'M4', 'S2', 'I2', 'M12', 'P8', 'P5']:
        test_features[f'{col}_lag1'] = test_features.get(col, pd.Series(0)).shift(1).ffill().bfill().fillna(0)
    test_features['forward_returns_lag1'] = test['lagged_forward_returns']
    test_features['market_forward_excess_returns_lag1'] = test['lagged_market_forward_excess_returns']
    test_features['V13_M1_interaction'] = test_features['V13'] * test_features['M1']
    test_features['S5_M4_interaction'] = test_features['S5'] * test_features['M4']
    test_features['V13_S5_interaction'] = test_features['V13'] * test_features['S5']
    test_features['V13_roll_mean'] = test_features['V13'].rolling(window=5).mean().fillna(0)
    test_features['S5_roll_mean'] = test_features['S5'].rolling(window=5).mean().fillna(0)
    test_features['V13_roll_std'] = test_features['V13'].rolling(window=5).std().fillna(0)
    test_features['M4_roll_mean'] = test_features['M4'].rolling(window=5).mean().fillna(0)
    test_features['S2_roll_mean'] = test_features['S2'].rolling(window=5).mean().fillna(0)
    test_features['I2_roll_mean'] = test_features['I2'].rolling(window=5).mean().fillna(0)
    test_features['M12_roll_mean'] = test_features['M12'].rolling(window=5).mean().fillna(0)
    test_features['P8_roll_mean'] = test_features['P8'].rolling(window=5).mean().fillna(0)
    test_features['P5_roll_mean'] = test_features['P5'].rolling(window=5).mean().fillna(0)
    test_features = test_features[feature_cols].ffill().bfill().fillna(0)

    for col in feature_cols:
        if col not in test_features.columns:
            test_features[col] = 0

    history_features = pd.concat([history_features, test_features])
    history_returns.append(test['lagged_forward_returns'][0])

    hist_len = len(history_features)
    seq_len = min(window, hist_len)
    seq_df = history_features.iloc[-seq_len:][feature_cols]
    if seq_len < window:
        pad_rows = window - seq_len
        pad_df = pd.DataFrame(np.zeros((pad_rows, len(feature_cols))), columns=feature_cols)
        seq_df = pd.concat([pad_df, seq_df])

    X_seq_scaled = scaler.transform(seq_df)
    X_seq_tensor = torch.tensor(X_seq_scaled, dtype=torch.float32).unsqueeze(0).to(device)
    X_rf_scaled = scaler.transform(test_features)

    model_lstm.eval()
    with torch.no_grad():
        prob_lstm = model_lstm(X_seq_tensor)[0].item()
        prob_lstm = min(max(prob_lstm, 1e-6), 1-1e-6)  
    prob_rf = model_rf.predict_proba(X_rf_scaled)[0][1]
    prob = lstm_weight * prob_lstm + rf_weight * prob_rf

    position = min(1.0, max(0.0, prob * 3.0)) if prob > 0.5 else 0.0

    return float(position)

In [ ]:
inference_server = kaggle_evaluation.default_inference_server.DefaultInferenceServer(predict)
if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/hull-tactical-market-prediction/',))